## 数据读取

In [1]:
file_proce_path = './Data/VersonData_2/traindf.csv'

In [2]:
import pandas as pd

In [3]:
traindf = pd.read_csv(file_proce_path,encoding='utf-8')

In [4]:
traindf.head()

,ID,Age,Gender,Education,Query List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...


## 制作word-emdeding ,使用tf-idf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tfidf_vec = TfidfVectorizer(analyzer='word',max_features=4000)

## 分类age

In [7]:
traindf['Age'].value_counts()

1    38996
2    26744
3    18529
4    10654
5     2922
0     1666
6      489
Name: Age, dtype: int64

In [8]:
agedf= traindf[['Age','Query List']]
agedf = agedf[agedf['Age']!=0]
agedf.head()

,Age,Query List
0,1,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,2,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,4,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,4,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...
4,2,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...


In [9]:
X_age = agedf['Query List'].values.tolist()
Y_age = agedf['Age'].values.tolist()

### 使用NB模型

In [10]:
from sklearn.naive_bayes import MultinomialNB

In [11]:
cls_age = MultinomialNB()

### 定义交叉验证函数  
cross_val_score 不知道怎么输入特征

In [12]:
from sklearn.model_selection import StratifiedKFold

In [13]:
def skfold_cv(x, y,vec_word, clf,shuffle=True, n_folds=5):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    history_score=[]
    vec_word.fit(x)
    for train_index, test_index in stratifiedk_fold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train,y_test = y[train_index],y[test_index]
        clf.fit(vec_word.transform(X_train),y_train)
        history_score.append(clf.score(vec_word.transform(X_test), y_test)) 
    return history_score 

In [16]:
import numpy as np

In [18]:
age_history_score = skfold_cv(np.array(X_age),np.array(Y_age),tfidf_vec,cls_age)

D:\PYTHON\SDK\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
print(age_history_score)

[0.5361228328842341, 0.5396074842383567, 0.5386179895256012, 0.5476686835816342, 0.5437115394395565]


## 分类性别

In [21]:
traindf['Gender'].value_counts()

1    56976
2    40869
0     2155
Name: Gender, dtype: int64

In [22]:
genderdf = traindf[['Age','Query List','Gender']]
genderdf = genderdf[genderdf['Gender']!= 0]

In [23]:
X_gender = genderdf['Query List'].values.tolist()
Y_gender = genderdf['Gender'].values.tolist() 
Y_ageflg = genderdf['Age'].values.tolist() 

In [24]:
def skfold_gender_cv(x, y,y_flag,vec_word, clf,shuffle=True, n_folds=5):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    history_score=[]
    vec_word.fit(x)
    for train_index, test_index in stratifiedk_fold.split(x, y_flag):
        X_train, X_test = x[train_index], x[test_index]
        y_train,y_test = y[train_index],y[test_index]
        clf.fit(vec_word.transform(X_train),y_train)
        history_score.append(clf.score(vec_word.transform(X_test), y_test)) 
    return history_score 

In [25]:
cls_gender = MultinomialNB()

In [26]:
gender_history_score = skfold_gender_cv(np.array(X_gender),np.array(Y_gender),np.array(Y_ageflg),tfidf_vec,cls_gender)

In [27]:
print(gender_history_score)

[0.801859799713877, 0.8048132440856369, 0.8016250191629618, 0.8042622783257526, 0.7919860983338445]


## 分类学历

In [28]:
traindf['Education'].value_counts()

5    37107
4    28148
3    18858
0     9280
6     5693
2      560
1      354
Name: Education, dtype: int64

In [29]:
educationdf = traindf[['Education','Query List']]
educationdf = educationdf[educationdf['Education']!=0]

In [30]:
X_education = educationdf['Query List'].values.tolist()
Y_education = educationdf['Education'].values.tolist() 

In [31]:
cls_education = MultinomialNB()

In [33]:
education_history_score = skfold_cv(np.array(X_education),np.array(Y_education),tfidf_vec,cls_education)

In [34]:
print(education_history_score)

[0.5755538410669018, 0.5687203791469194, 0.5678699366216589, 0.5641605115202293, 0.5654043327269721]


## 求平均率

In [35]:
print('age score  ',np.mean(age_history_score ))
print('gender score  ',np.mean(gender_history_score))
print('education score  ',np.mean(education_history_score))

age score   0.5411457059338767
gender score   0.8009092879244146
education score   0.5683418002165362


In [36]:
average_score = (np.mean(age_history_score) + np.mean(gender_history_score)+ np.mean(education_history_score) )/3

In [37]:
average_score

0.6367989313582759